In [1]:
import pandas as pd
import numpy as np

In [2]:
# 출력 옵션 변경

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1300)

In [3]:
dlvr_raw = pd.read_csv('../data/dlvr_call.csv');

c:\users\jeaung-lee\appdata\local\programs\python\python39\lib\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [13]:
dlvr = dlvr_raw.copy()

In [14]:
dlvr.shape

(4296271, 32)

In [15]:
dlvr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4296271 entries, 0 to 4296270
Data columns (total 32 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   SN                         int64 
 1   PROCESS_DT                 object
 2   DLVR_REQUST_ENTRPS_ID      object
 3   DLVR_REQUST_STTUS_VALUE    int64 
 4   DLVR_RCEPT_ENTRPS_ID       object
 5   DLVR_STORE_ID              object
 6   DLVR_STORE_INDUTY_NM       object
 7   DLVR_STORE_LEGALDONG_CODE  int64 
 8   DLVR_STORE_BRTC_NM         object
 9   DLVR_STORE_SIGNGU_NM       object
 10  DLVR_STORE_LEGALDONG_NM    object
 11  DLVR_STORE_LI_NM           object
 12  DLVR_STORE_ADSTRD_CODE     int64 
 13  DLVR_STORE_RDNMADR_CODE    object
 14  DLVR_DSTN_LEGALDONG_CODE   int64 
 15  DLVR_DSTN_BRTC_NM          object
 16  DLVR_DSTN_SIGNGU_NM        object
 17  DLVR_DSTN_LEGALDONG_NM     object
 18  DLVR_DSTN_LI_NM            object
 19  DLVR_DSTN_ADSTRD_CODE      object
 20  DLVR_DSTN_RDNMADR_CODE  

In [16]:
# 하나의 구 추출

dlvr = dlvr[dlvr['DLVR_STORE_SIGNGU_NM'] == '구로구']

In [17]:
dlvr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357818 entries, 9 to 4296224
Data columns (total 32 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   SN                         357818 non-null  int64 
 1   PROCESS_DT                 357818 non-null  object
 2   DLVR_REQUST_ENTRPS_ID      357818 non-null  object
 3   DLVR_REQUST_STTUS_VALUE    357818 non-null  int64 
 4   DLVR_RCEPT_ENTRPS_ID       357818 non-null  object
 5   DLVR_STORE_ID              357818 non-null  object
 6   DLVR_STORE_INDUTY_NM       357818 non-null  object
 7   DLVR_STORE_LEGALDONG_CODE  357818 non-null  int64 
 8   DLVR_STORE_BRTC_NM         357818 non-null  object
 9   DLVR_STORE_SIGNGU_NM       357818 non-null  object
 10  DLVR_STORE_LEGALDONG_NM    357818 non-null  object
 11  DLVR_STORE_LI_NM           357818 non-null  object
 12  DLVR_STORE_ADSTRD_CODE     357818 non-null  int64 
 13  DLVR_STORE_RDNMADR_CODE    357818 non-null 

In [18]:
# 배달일시, 지역, 업종, 매출 컬럼 선택
# 배달일시 ['PROCESS_DT'], 행정동코드 ['DLVR_DSTN_ADSTRD_CODE'], 배달상점 업종이름 (카테고리) ['DLVR_STORE_INDUTY_NM'], 배달상품금액 ['GOODS_AMOUNT']

df_dlvr = dlvr[['PROCESS_DT','DLVR_STORE_SIGNGU_NM','DLVR_STORE_INDUTY_NM','GOODS_AMOUNT']]

In [19]:
# 업종 컬럼 값 확인

df_dlvr['DLVR_STORE_INDUTY_NM'].unique()

array(['패스트푸드', '분식', '회', '치킨', '피자', '족발/보쌈', '카페/디저트', '한식', '중식',
       '야식', '찜탕', '돈까스/일식', '도시락', '배달전문업체', '아시안/양식'], dtype=object)

In [20]:
# 업종이'배달전문업체','심부름'인 행 삭제

df_dlvr.replace(['배달전문업체','심부름'], np.nan, inplace=True)
df_dlvr = df_dlvr.dropna(axis=0) 

c:\users\jeaung-lee\appdata\local\programs\python\python39\lib\site-packages\pandas\core\frame.py:4524: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [23]:
# 처리일시를 날짜 데이터 형태로 변경
df_dlvr['PROCESS_DT'] = pd.to_datetime(df_dlvr['PROCESS_DT'])

In [28]:
df_dlvr = df_dlvr.sort_values(by='PROCESS_DT', ascending=True)

In [78]:
date = df_dlvr['PROCESS_DT'].unique()
df = pd.DataFrame()
df_week = pd.DataFrame(columns=['업종'])
cnt = 0

for d in date:
    cnt += 1
    df_date = df_dlvr[df_dlvr['PROCESS_DT'] == d]
    induty = df_date['DLVR_STORE_INDUTY_NM'].unique()    
    for i in induty:
        df_induty = df_date[df_date['DLVR_STORE_INDUTY_NM'] == i]
        indSum = sum(df_induty['GOODS_AMOUNT'])
        dic = { '업종' : i , '매출' : indSum}
        df_week = df_week.append(dic,ignore_index=True)

    if cnt % 7 == 0:
        df_week_sum = df_week.groupby('업종', sort=False).sum()
        df_week_sum['week'] = round(cnt/7)
        df = pd.concat([df,df_week_sum],axis=0)
        df_week = pd.DataFrame(columns=['업종'])
    else:
            pass
        

In [80]:
df

,매출,week
업종,,
패스트푸드,31402200.0,1
아시안/양식,126500.0,1
도시락,1507200.0,1
돈까스/일식,1908700.0,1
찜탕,3649900.0,1
야식,6337000.0,1
한식,22154500.0,1
중식,6496000.0,1
족발/보쌈,22048000.0,1


In [81]:
df1 = df.copy() 

In [82]:
df1

,매출,week
업종,,
패스트푸드,31402200.0,1
아시안/양식,126500.0,1
도시락,1507200.0,1
돈까스/일식,1908700.0,1
찜탕,3649900.0,1
야식,6337000.0,1
한식,22154500.0,1
중식,6496000.0,1
족발/보쌈,22048000.0,1


In [83]:
# 정수형으로 변경(데이터 표기 통일)

df1.replace(np.nan, 0, inplace=True)
df1 = df1.astype('int64')
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362 entries, 패스트푸드 to 회
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   매출      362 non-null    int64
 1   week    362 non-null    int64
dtypes: int64(2)
memory usage: 8.5+ KB


In [84]:
df1.to_csv('../data/weeklySales.csv')